# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [117]:
import pandas as pd
import numpy as np
import requests
from scipy import stats

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [118]:
stocks = pd.read_csv("constituents.csv")
from secrets_api import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [119]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 3078822785920,
 'week52high': 198,
 'week52low': 123.15,
 'week52highSplitAdjustOnly': 198.23,
 'week52lowSplitAdjustOnly': 124.17,
 'week52change': 0.395501629119148,
 'sharesOutstanding': 15552752000,
 'float': 0,
 'avg10Volume': 56092419,
 'avg30Volume': 48000708,
 'day200MovingAvg': 182.5,
 'day50MovingAvg': 185.79,
 'employees': 164000,
 'ttmEPS': 6.13,
 'ttmDividendRate': 0.9465536932059656,
 'dividendYield': 0.004781540175823225,
 'nextDividendDate': '',
 'exDividendDate': '2023-11-10',
 'nextEarningsDate': '2024-02-01',
 'peRatio': 31.74207728150936,
 'beta': 1.171879306839512,
 'maxChangePercent': 76.37648530331458,
 'year5ChangePercent': 3.844942742253256,
 'year2ChangePercent': 0.14379018880396033,
 'year1ChangePercent': 0.3738216058361741,
 'ytdChangePercent': 0.5361955801396514,
 'month6ChangePercent': 0.08569212873206689,
 'month3ChangePercent': 0.13932501496385652,
 'month1ChangePercent': 0.07121212121212128,
 'day30ChangePercen

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [120]:
data['year1ChangePercent']

0.3738216058361741

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [121]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Symbol'].sort_values(), 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

for symbol_string in symbol_strings:
    print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,AON,AOS,APA,APD,APH,APTV,ARE,ATO,AVB,AVGO,AVY,AWK,AXON,AXP,AZO,BA,BAC,BALL,BAX,BBWI,BBY,BDX,BEN,BF.B,BG,BIIB,BIO,BK,BKNG,BKR,BLK,BMY,BR,BRK.B,BRO,BSX,BWA,BX,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDAY,CDNS,CDW,CE,CEG,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA
CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COO,COP,COR,COST,CPB,CPRT,CPT,CRL,CRM,CSCO,CSGP,CSX,CTAS,CTLT,CTRA,CTSH,CTVA,CVS,CVX,CZR,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DLR,DLTR,DOV,DOW,DPZ,DRI,DTE,DUK,DVA,DVN,DXCM,EA,EBAY,ECL,ED,EFX,EG,EIX,EL,ELV,EMN,EMR,ENPH,EOG,EPAM,EQIX,EQR,EQT,ES,ESS,ETN,ETR,ETSY,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FCX,FDS,FDX,FE,FFIV,FI,FICO,FIS,FITB,FLT,FMC,FOX,FOXA,FRT,FSLR,FTNT,FTV,GD,GE,GEHC
GEN,GILD,GIS,GL,GLW,GM,GNRC,GOOG,GOOGL,GPC,GPN,GRMN,GS,GWW,HAL,HAS,HBAN,HCA,HD,HES,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRL,HSIC,HST,HSY,HUBB,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [122]:
my_df = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    #print(data)
    for symbol in symbol_string.split(','):
        new_row = pd.DataFrame(
            [symbol, data[symbol]['quote']['latestPrice'],data[symbol]['stats']['year1ChangePercent'], 'NaN' ],
            index = my_columns,
        ).transpose()
        my_df = pd.concat([my_df, new_row],ignore_index=True)
my_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,137.1,-0.145726,NaN
1,AAL,14.54,0.046805,NaN
2,AAPL,196.655,0.373822,NaN
3,ABBV,154.6,-0.02583,NaN
4,ABNB,146.34,0.548211,NaN
...,...,...,...,...
498,YUM,129.51,0.024921,NaN
499,ZBH,118.715,-0.074957,NaN
500,ZBRA,266.06,-0.071562,NaN
501,ZION,45.425,-0.090439,NaN


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [123]:
final_df = my_df.sort_values(by='One-Year Price Return', ascending= False)
final_df = final_df[:50]
final_df.reset_index(inplace=True)
final_df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,302,META,330.74,1.786017,NaN
1,342,NVDA,476.32,1.662093,NaN
2,390,RCL,121.605,1.160509,NaN
3,370,PHM,103.235,1.154506,NaN
4,84,CCL,18.72,1.050505,NaN
5,45,AVGO,1113.14,0.969402,NaN
6,356,PANW,299.16,0.952473,NaN
7,28,AMD,136.385,0.928681,NaN
8,116,CRM,255.14,0.89736,NaN
9,186,FICO,1135.05,0.880205,NaN


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [124]:
def portfolio_input():
    global portfolio
    try:
        portfolio = float(input("Please input your portfolio:"))
    except ValueError:
        print("Please enter an number")
        portfolio = float(input("Please input your portfolio:"))

portfolio_input()
final_df['Number of Shares to Buy'] = final_df.apply(lambda x: int((portfolio/len(x.index)//x['Price'])), axis=1)

In [125]:
final_df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,302,META,330.74,1.786017,604
1,342,NVDA,476.32,1.662093,419
2,390,RCL,121.605,1.160509,1644
3,370,PHM,103.235,1.154506,1937
4,84,CCL,18.72,1.050505,10683
5,45,AVGO,1113.14,0.969402,179
6,356,PANW,299.16,0.952473,668
7,28,AMD,136.385,0.928681,1466
8,116,CRM,255.14,0.89736,783
9,186,FICO,1135.05,0.880205,176


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [126]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

#my_df = pd.DataFrame([[0]*12],columns=hqm_columns)
my_df = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    #print(data)
    for symbol in symbol_string.split(','):
        new_row = pd.DataFrame(
            [
                symbol, 
                data[symbol]['quote']['latestPrice'],
                'NaN',
                data[symbol]['stats']['year1ChangePercent'],
                'NaN',
                data[symbol]['stats']['month6ChangePercent'],
                'NaN',
                data[symbol]['stats']['month3ChangePercent'],
                'NaN',
                data[symbol]['stats']['month1ChangePercent'],
                'NaN',
                'NaN'
            ],
            index = hqm_columns,
        ).transpose()
        my_df = pd.concat([my_df, new_row],ignore_index=True)
my_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.1,NaN,-0.145726,NaN,0.123562,NaN,0.18143,NaN,0.244556,NaN,NaN
1,AAL,14.54,NaN,0.046805,NaN,-0.136113,NaN,0.058603,NaN,0.196095,NaN,NaN
2,AAPL,196.65,NaN,0.373822,NaN,0.085692,NaN,0.139325,NaN,0.071212,NaN,NaN
3,ABBV,154.63,NaN,-0.02583,NaN,0.139268,NaN,0.03099,NaN,0.112954,NaN,NaN
4,ABNB,146.34,NaN,0.548211,NaN,0.157006,NaN,-0.001417,NaN,0.212841,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,129.51,NaN,0.024921,NaN,-0.028051,NaN,0.019209,NaN,0.049791,NaN,NaN
499,ZBH,118.715,NaN,-0.074957,NaN,-0.13504,NaN,-0.020509,NaN,0.114156,NaN,NaN
500,ZBRA,266.06,NaN,-0.071562,NaN,-0.137478,NaN,-0.022983,NaN,0.206406,NaN,NaN
501,ZION,45.425,NaN,-0.090439,NaN,0.438369,NaN,0.171573,NaN,0.278508,NaN,NaN


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [127]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in my_df.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        my_df.loc[row,percentile_col] = stats.percentileofscore(my_df[change_col],my_df.loc[row,change_col])/100

my_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.1,NaN,-0.145726,0.135189,0.123562,0.652087,0.18143,0.872763,0.244556,0.940358,NaN
1,AAL,14.54,NaN,0.046805,0.475149,-0.136113,0.095427,0.058603,0.465209,0.196095,0.838966,NaN
2,AAPL,196.65,NaN,0.373822,0.87674,0.085692,0.54672,0.139325,0.753479,0.071212,0.333996,NaN
3,ABBV,154.63,NaN,-0.02583,0.328032,0.139268,0.699801,0.03099,0.365805,0.112954,0.5666,NaN
4,ABNB,146.34,NaN,0.548211,0.944334,0.157006,0.747515,-0.001417,0.246521,0.212841,0.882704,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,129.51,NaN,0.024921,0.441352,-0.028051,0.242545,0.019209,0.32008,0.049791,0.228628,NaN
499,ZBH,118.715,NaN,-0.074957,0.234592,-0.13504,0.097416,-0.020509,0.202783,0.114156,0.578529,NaN
500,ZBRA,266.06,NaN,-0.071562,0.238569,-0.137478,0.093439,-0.022983,0.194831,0.206406,0.868787,NaN
501,ZION,45.425,NaN,-0.090439,0.210736,0.438369,0.996024,0.171573,0.842942,0.278508,0.964215,NaN


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [128]:
from statistics import mean
my_df['HQM Score'] = my_df.apply(lambda x: mean([x[f'{time_period} Return Percentile'] for time_period in time_periods]),axis=1)
my_df

columns_to_mean = [f'{time_period} Return Percentile' for time_period in time_periods]
print(columns_to_mean)

my_df['HQM Score'] = my_df[columns_to_mean].mean(axis=1)
my_df

['One-Year Return Percentile', 'Six-Month Return Percentile', 'Three-Month Return Percentile', 'One-Month Return Percentile']


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.1,NaN,-0.145726,0.135189,0.123562,0.652087,0.18143,0.872763,0.244556,0.940358,0.650099
1,AAL,14.54,NaN,0.046805,0.475149,-0.136113,0.095427,0.058603,0.465209,0.196095,0.838966,0.468688
2,AAPL,196.65,NaN,0.373822,0.87674,0.085692,0.54672,0.139325,0.753479,0.071212,0.333996,0.627734
3,ABBV,154.63,NaN,-0.02583,0.328032,0.139268,0.699801,0.03099,0.365805,0.112954,0.5666,0.490060
4,ABNB,146.34,NaN,0.548211,0.944334,0.157006,0.747515,-0.001417,0.246521,0.212841,0.882704,0.705268
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,129.51,NaN,0.024921,0.441352,-0.028051,0.242545,0.019209,0.32008,0.049791,0.228628,0.308151
499,ZBH,118.715,NaN,-0.074957,0.234592,-0.13504,0.097416,-0.020509,0.202783,0.114156,0.578529,0.278330
500,ZBRA,266.06,NaN,-0.071562,0.238569,-0.137478,0.093439,-0.022983,0.194831,0.206406,0.868787,0.348907
501,ZION,45.425,NaN,-0.090439,0.210736,0.438369,0.996024,0.171573,0.842942,0.278508,0.964215,0.753479


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [129]:
sorted_df = my_df.sort_values(by='HQM Score', ascending=False)[:50]
sorted_df.reset_index(inplace=True, drop= True)
sorted_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,PANW,299.09,NaN,0.952473,0.988072,0.343406,0.978131,0.274351,0.978131,0.227742,0.912525,0.964215
1,LULU,490.25,NaN,0.546265,0.942346,0.369584,0.986083,0.297015,0.986083,0.20989,0.878728,0.948310
2,RCL,121.605,NaN,1.160509,0.996024,0.257516,0.908549,0.237494,0.956262,0.231674,0.922465,0.945825
3,EXPE,146.36,NaN,0.53512,0.93837,0.305876,0.94831,0.356488,0.998012,0.222991,0.894632,0.944831
4,PHM,103.235,NaN,1.154506,0.994036,0.33134,0.966203,0.275466,0.980119,0.194052,0.833002,0.943340
5,STX,86.56,NaN,0.628662,0.960239,0.379013,0.99006,0.376786,1.0,0.178833,0.805169,0.938867
6,CCL,18.71,NaN,1.050505,0.992048,0.191004,0.809145,0.216378,0.924453,0.429577,0.998012,0.930915
7,FICO,1135.05,NaN,0.880205,0.982107,0.4556,1.0,0.300091,0.99006,0.152637,0.73161,0.925944
8,LEN,153.5,NaN,0.624142,0.95825,0.259234,0.910537,0.255868,0.968191,0.187403,0.817097,0.913519
9,DHI,151.84,NaN,0.66689,0.964215,0.250669,0.900596,0.269177,0.976143,0.181398,0.809145,0.912525


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [130]:
portfolio_input()
sorted_df['Number of Shares to Buy'] = sorted_df.apply(lambda x: int((portfolio/len(x.index)//x['Price'])), axis=1)

In [131]:
sorted_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,PANW,299.09,278,0.952473,0.988072,0.343406,0.978131,0.274351,0.978131,0.227742,0.912525,0.964215
1,LULU,490.25,169,0.546265,0.942346,0.369584,0.986083,0.297015,0.986083,0.20989,0.878728,0.948310
2,RCL,121.605,685,1.160509,0.996024,0.257516,0.908549,0.237494,0.956262,0.231674,0.922465,0.945825
3,EXPE,146.36,569,0.53512,0.93837,0.305876,0.94831,0.356488,0.998012,0.222991,0.894632,0.944831
4,PHM,103.235,807,1.154506,0.994036,0.33134,0.966203,0.275466,0.980119,0.194052,0.833002,0.943340
5,STX,86.56,962,0.628662,0.960239,0.379013,0.99006,0.376786,1.0,0.178833,0.805169,0.938867
6,CCL,18.71,4453,1.050505,0.992048,0.191004,0.809145,0.216378,0.924453,0.429577,0.998012,0.930915
7,FICO,1135.05,73,0.880205,0.982107,0.4556,1.0,0.300091,0.99006,0.152637,0.73161,0.925944
8,LEN,153.5,542,0.624142,0.95825,0.259234,0.910537,0.255868,0.968191,0.187403,0.817097,0.913519
9,DHI,151.84,548,0.66689,0.964215,0.250669,0.900596,0.269177,0.976143,0.181398,0.809145,0.912525


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [141]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
sorted_df.to_excel(writer,sheet_name='Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [142]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [143]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', percent_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}',25,column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1',column_formats[column][0],string_template)


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [144]:
writer.save()

/var/folders/mj/9ptdvvyj6vd_d80g6y817crw0000gn/T/ipykernel_34050/934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
